In [61]:
import cv2
import numpy as np
import random

In [62]:
# Configurations

CFG = {
    'ratio': 0.85,
    'min_match': 10,
    'smoothing_window_size': 100,
    'estimation_thresh': 0.60
}

In [63]:
# Runs sift algorithm to find features

def findFeatures(img):
    sift = cv2.xfeatures2d.SIFT_create()
    keypoints, descriptors = sift.detectAndCompute(img, None)

    return keypoints, descriptors

In [64]:
# Matches features given a list of keypoints, descriptors, and images

def matchFeatures(desc1, desc2):
    matcher = cv2.BFMatcher()
    matches = matcher.knnMatch(desc1, desc2, k=2)
    
    return matches

In [65]:
# Computers a homography from 4-correspondences
# referred from: https://www.youtube.com/watch?v=l_qjO4cM74o&t=573s

def calculateHomography(correspondences):
    # loop through correspondences and create assemble matrix
    aList = []
    for corr in correspondences:
        p1 = np.matrix([corr.item(0), corr.item(1), 1])
        p2 = np.matrix([corr.item(2), corr.item(3), 1])

        a1 = [p1.item(0), p1.item(1), 1, 0, 0, 0, -p2.item(0)*p1.item(0), -p2.item(0)*p1.item(1), -p2.item(0)]
        a2 = [0, 0, 0, p1.item(0), p1.item(1), 1, -p2.item(1)*p1.item(0), -p2.item(1)*p1.item(1), -p2.item(1)]
        
        aList.append(a1)
        aList.append(a2)

    matrixA = np.matrix(aList)

    # svd composition
    u, s, v = np.linalg.svd(matrixA)

    # reshape the min singular value into a 3 by 3 matrix
    h = np.reshape(v[8], (3, 3))

    # normalize
    h = (1/h.item(8)) * h
    return h

In [66]:
# Calculate the geometric distance (error) between estimated points and original points

def geometricDistance(correspondence, h):

    p1 = np.transpose(np.matrix([correspondence[0].item(0), correspondence[0].item(1), 1]))
    estimatep2 = np.dot(h, p1)
    estimatep2 = (1/estimatep2.item(2))*estimatep2

    p2 = np.transpose(np.matrix([correspondence[0].item(2), correspondence[0].item(3), 1]))
    error = p2 - estimatep2
    return np.linalg.norm(error)

In [67]:
# Runs through ransac algorithm, creating homographies from random correspondences

def ransac(corr, thresh):
    maxInliers = []
    finalH = None
    for i in range(1000):
        # find 4 random points to calculate a homography
        randomFour = corr[random.randrange(0, len(corr))]
        for j in range(3):
            corr_add = corr[random.randrange(0, len(corr))]
            randomFour = np.vstack((randomFour, corr_add))

        h = calculateHomography(randomFour)
        
        inliers = []
        error_list = []
        for i in range(len(corr)):
            d = geometricDistance(corr[i], h)
            error_list.append(d)
            if d < 5:
                inliers.append(corr[i])

        if len(inliers) > len(maxInliers):
            maxInliers = inliers
            finalH = h
        print("Corr size: ", len(corr), " NumInliers: ", len(inliers), "Max inliers: ", len(maxInliers))

        if len(maxInliers) > (len(corr)*thresh):
            break

    return finalH, maxInliers, error_list

In [68]:
def create_mask(img1, img2, version):
    height_img1 = img1.shape[0]
    width_img1 = img1.shape[1]
    width_img2 = img2.shape[1]
    
    height_panorama = height_img1
    width_panorama = width_img1 + width_img2
    offset = int(CFG['smoothing_window_size'] / 2)
    barrier = img1.shape[1] - int(CFG['smoothing_window_size'] / 2)
    mask = np.zeros((height_panorama, width_panorama))
    
    if version== 'left_image':
        mask[:, barrier - offset:barrier + offset ] = np.tile(np.linspace(1, 0, 2 * offset ).T, (height_panorama, 1))
        mask[:, :barrier - offset] = 1
    else:
        mask[:, barrier - offset :barrier + offset ] = np.tile(np.linspace(0, 1, 2 * offset ).T, (height_panorama, 1))
        mask[:, barrier + offset:] = 1
    return cv2.merge([mask, mask, mask])

In [69]:
def blending(img1, img2, homography):
    height_img1 = img1.shape[0]
    width_img1 = img1.shape[1]
    width_img2 = img2.shape[1]
    height_panorama = height_img1
    width_panorama = width_img1 + width_img2

    mask1 = create_mask(img1, img2, version='left_image')
    panorama1 = np.zeros((height_panorama, width_panorama, 3))
    panorama1[0:img1.shape[0], 0:img1.shape[1], :] = img1
    panorama1 *= mask1
    
    mask2 = create_mask(img1, img2, version='right_image')
    panorama2 = cv2.warpPerspective(img2, homography, (width_panorama, height_panorama))*mask2
    
    result = panorama1 + panorama2

    rows, cols = np.where(result[:, :, 0] != 0)
    min_row, max_row = min(rows), max(rows) + 1
    min_col, max_col = min(cols), max(cols) + 1
    final_result = result[min_row:max_row, min_col:max_col, :]
    return final_result, panorama1, panorama2

In [70]:
img1 = cv2.imread('./img/img_1111.jpg') # query image
img2 = cv2.imread('./img/img_2222.jpg') # train image_1
img3 = cv2.imread('./img/img_3333.jpg') # train image_2

list_img = []
list_img.appned(img1)
list_img.appned(img2)
list_img.appned(img3)

In [ ]:
for img in list_img:
    

In [71]:
for idx in range(len(list_img)):
    if img is not None:
        kp1, desc1 = findFeatures(img)






    #find features and keypoints
    correspondenceList = []
    if img1 is not None and img2 is not None:
        # find features
        kp1, desc1 = findFeatures(img1)
        kp2, desc2 = findFeatures(img2)
        print("Found keypoints in " + 'img1' + ": " + str(len(kp1)))
        print("Found keypoints in " + 'img2' + ": " + str(len(kp2)))
        keypoints = [kp1, kp2]
        
        # match features 
        matches = matchFeatures(desc1, desc2)
        
        good_points = []
        good_matches=[]
        for m1, m2 in matches:
            if m1.distance < CFG['ratio'] * m2.distance:
                good_points.append((m1.trainIdx, m1.queryIdx))
                good_matches.append([m1])
        
        img3 = cv2.drawMatchesKnn(img1, kp1, img2, kp2, good_matches, None, flags=2)
        cv2.imwrite('matching.jpg', img3)
        
        if len(good_points) > CFG['min_match']:       
            for (i, j) in good_points:
                (x1, y1) = kp1[j].pt
                (x2, y2) = kp2[i].pt
                correspondenceList.append([x2, y2, x1, y1])

        corrs = np.matrix(correspondenceList)

        # find homography using ransac algorithm
        finalH, inliers, error_list = ransac(corrs, CFG['estimation_thresh'])
        print("Final homography: ", finalH)
        print("Final inliers count: ", len(inliers))

        f = open('homography.txt', 'w')
        f.write("Final homography: \n" + str(finalH)+"\n")
        f.write("Final inliers count: " + str(len(inliers)))
        f.close()
    else:
        print("Failed to load images")

Found keypoints in img1: 18618
Found keypoints in img2: 12081
Corr size:  3579  NumInliers:  2146 Max inliers:  2146
Corr size:  3579  NumInliers:  960 Max inliers:  2146
Corr size:  3579  NumInliers:  6 Max inliers:  2146
Corr size:  3579  NumInliers:  5 Max inliers:  2146
Corr size:  3579  NumInliers:  1111 Max inliers:  2146
Corr size:  3579  NumInliers:  13 Max inliers:  2146
Corr size:  3579  NumInliers:  5 Max inliers:  2146
Corr size:  3579  NumInliers:  5 Max inliers:  2146
Corr size:  3579  NumInliers:  1579 Max inliers:  2146
Corr size:  3579  NumInliers:  11 Max inliers:  2146
Corr size:  3579  NumInliers:  1361 Max inliers:  2146
Corr size:  3579  NumInliers:  2417 Max inliers:  2417
Final homography:  [[ 5.02572133e-01 -2.85981138e-02  5.19894482e+02]
 [-1.35733174e-01  8.49962053e-01  5.28621483e+01]
 [-3.79981011e-04  1.79658399e-05  1.00000000e+00]]
Final inliers count:  2417


In [72]:
output, pano1, pano2 = blending(img1, img2, finalH)

cv2.imwrite('pano1.jpg', pano1)
cv2.imwrite('pano2.jpg', pano2)
cv2.imwrite('panorama.jpg', output)

True